In [25]:
import datetime
import pandas as pd
import numpy as np
from btyd import BetaGeoModel

PATH = 'data\\wallet_urfu.parquet.gzip'

In [ ]:
# partner — идентификатор клиента
# rep_date — дата пополнения. Запись в кошелек выполняется ежедневно. 
# Нулевые пополнения не записываются. Если у клиента поступления в некоторую дату нет, то запись не производится.
# monetary — сумма пополнения. Значения приведены к интервалу 0 — 1.

In [3]:
data = pd.read_parquet(PATH)
data

,partner,rep_date,monetary
0,468726,2016-01-02,2.673749e-11
1,134145,2016-01-02,1.069500e-12
2,66774,2016-01-02,2.228576e-12
3,178928,2016-01-02,8.730611e-11
4,679693,2016-01-02,3.492244e-11
...,...,...,...
52032794,636021,2023-02-23,4.365305e-12
52032795,1408806,2023-02-23,5.896654e-12
52032796,360238,2023-02-23,4.365305e-12
52032797,1007996,2023-02-23,6.547958e-13


In [ ]:
rfm = pd.concat([
    data.groupby('partner', as_index=True).agg(monetary_value=("monetary", np.mean)),
    data.groupby('partner', as_index=True).agg(first_buy=("rep_date", np.min)),
    data.groupby('partner', as_index=True).agg(last_buy=("rep_date", np.max)),
    data.groupby('partner', as_index=True).agg(count=('partner', np.size))],
axis=1)

In [48]:
last_data_date = datetime.date(day=23, year=2023, month=2)

rfm['frequency'] = rfm['count'] - 1
rfm['recency'] = rfm['last_buy'] - rfm['first_buy']
rfm['T'] = last_data_date - rfm['first_buy']

In [55]:
rfm

,monetary_mean,first_buy,last_buy,count,freq,recency,T
partner,,,,,,,
0,2.852146e-10,2016-01-27,2023-02-15,138,137,2576 days,2584 days
1,1.468186e-10,2016-01-15,2023-02-22,267,266,2595 days,2596 days
2,2.942489e-10,2016-03-04,2023-02-01,58,57,2525 days,2547 days
3,1.451897e-08,2016-04-01,2022-07-13,7,6,2294 days,2519 days
4,1.302467e-10,2016-01-04,2022-12-26,179,178,2548 days,2607 days
...,...,...,...,...,...,...,...
1695379,1.126811e-12,2016-07-26,2019-09-20,4,3,1151 days,2403 days
1695380,1.513636e-08,2016-03-22,2017-06-19,3,2,454 days,2529 days
1695381,1.341880e-09,2016-09-06,2022-02-25,23,22,1998 days,2361 days
